In [24]:
#importando bibliotecas

%matplotlib inline
import matplotlib
import os


import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [25]:
#!echo "Baixando 101_Object_Categories for image notebooks"
#!curl -L -o 101_ObjectCategories.tar.gz --progress-bar https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip?download=1
#!unzip 101_ObjectCategories.tar.gz
#!tar -xzf 101_ObjectCategories.tar.gz
#!rm 101_ObjectCategories.tar.gz
#!ls


In [26]:
!echo "Baixando 101_Object_Categories for image notebooks"
!curl -L -o dio-desafio-transfer-learning-main.zip --progress-bar https://github.com/lucasdionis10/dio-desafio-transfer-learning/archive/refs/heads/main.zip
!unzip dio-desafio-transfer-learning-main.zip
!ls
#!tar --help


Baixando 101_Object_Categories for image notebooks
######################################################################## 100.0%
Archive:  dio-desafio-transfer-learning-main.zip
05c87675178f9d8108bc63e6e43646cd4bdff20d
replace dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0002.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0002.jpg  
 extracting: dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0003.jpg  
  inflating: dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0004.jpg  
  inflating: dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0005.jpg  
  inflating: dio-desafio-transfer-learning-main/101_ObjectCategories/anchor/image_0006.jpg  
  inflating: dio-desafio-transfer-learning-main/101_ObjectCa

In [36]:
#mostrando as categorias
root = '101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

[]


In [28]:
# carregando imagens em vetores
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [29]:
#carregando as imagens da pasta root
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [30]:
#randomizando as imagens
random.shuffle(data)

In [35]:
# criando uma validação/treino (70%, 15%, 15%)
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [34]:
# separando os dados por rótulos
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

[]


In [33]:
# normalizando as informações
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convertendo os rótulos em vetores
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

ValueError: ignored

In [ ]:
# apresentando os resultados:
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)

In [ ]:
#testando
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

In [ ]:
# Construindo uma network

model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

In [ ]:
# compilando o modelo para usar entropia entre categorias 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))

In [ ]:
#plotando gráficos de perdas e precisão ao longo do tempo
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
#testando perdas e precisão
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
#trabalhando com vgg
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

In [ ]:
# referenciando as camadas de entrada do vgg
inp = vgg.input

# fazendo uma nova softmax layer com neurônios num_classes
new_classification_layer = Dense(num_classes, activation='softmax')

# conectando a nova camada da segunda à ultima camada no VGG, e referenciando
out = new_classification_layer(vgg.layers[-2].output)

# criando uma nova network entre inp e out
model_new = Model(inp, out)

In [ ]:
# fazendo todas as  camadas não treinadas com pesos congelados (menos a última)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False
# garantindo que a última camada seja treinável/não congelada
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
history2 = model_new.fit(x_train, y_train, 
                         batch_size=128, 
                         epochs=10, 
                         validation_data=(x_val, y_val))


In [ ]:
#plotando novamente a precisão e a perda
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.plot(history2.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
#novo teste
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
#previsão de novas imagens
img, x = get_image('101_ObjectCategories/airplanes/image_0003.jpg')
probabilities = model_new.predict([x])
